In [ ]:
import json
from ast import literal_eval
from envs.tool_manager.qwen3_manager import QwenManager


tools = QwenManager().initConfig(tool)

TypeError: QwenManager.__init__() missing 1 required positional argument: 'verl_config'

In [2]:
example_data = {"id": "multi_turn_miss_func_0", "question": [[{"role": "user", "content": "Move 'final_report.pdf' within document directory to 'temp' directory in document. Make sure to create the directory"}], [{"role": "user", "content": "Perform a detailed search using grep to identify sections in the file pertaining to 'budget analysis'."}], [{"role": "user", "content": "Upon identifying the requisite 'budget analysis' content, sort the 'final_report.pdf' by line for improved clarity and comprehension."}], [], [{"role": "user", "content": "Move 'previous_report.pdf' in document directory to temp as well and having final report also there, proceed to juxtapose it with 'previous_report.pdf' to detect any critical alterations."}]], "initial_config": {"GorillaFileSystem": {"root": {"workspace": {"type": "directory", "contents": {"document": {"type": "directory", "contents": {"final_report.pdf": {"type": "file", "content": "Year2024 This is the final report content including budget analysis and other sections."}, "previous_report.pdf": {"type": "file", "content": "Year203 This is the previous report content with different budget analysis."}}}, "archive": {"type": "directory", "contents": {}}}}}}, "TwitterAPI": {"tweet_counter": 3, "tweets": {"0": {"id": 0, "username": "analyst_pro", "content": "Just finished analyzing the reports!", "tags": ["#analysis", "#reports"], "mentions": []}, "1": {"id": 1, "username": "analyst_pro", "content": "Budget analysis insights coming soon!", "tags": ["#budget", "#analysis", "#insights"], "mentions": []}, "2": {"id": 2, "username": "analyst_pro", "content": "Stay tuned for more updates!", "tags": ["#updates", "#staytuned"], "mentions": []}}, "username": "analyst_pro", "password": "Kj8#mP9$vL2"}}, "path": ["GorillaFileSystem.find", "GorillaFileSystem.mv", "GorillaFileSystem.grep", "GorillaFileSystem.sort", "GorillaFileSystem.diff", "TwitterAPI.post_tweet"], "involved_classes": ["TwitterAPI", "GorillaFileSystem"], "missed_function": {"3": ["sort"]}}

In [3]:
tools[0].call(
    params = json.dumps({
        "scenario": example_data['initial_config']['GorillaFileSystem']
    })
)

'Sucessfully load from scenario.'